# Install Earth Engine API and gee
This code checks if the `geemap` package has been installed, and if not it will install `geemap` and its dependencies including earthengine-api, ipyleaflet, and folium.

Resources: https://github.com/giswqs/geemap and https://github.com/giswqs/earthengine-py-notebooks by Qiusheng Wu 


In [2]:
import geemap
# Authenticates and initializes Earth Engine
import ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()


# Set bounding box for map and image collection


In [3]:
region = ee.Geometry.BBox(-105.89218, 40.27989, -105.17284, 40.72316)

# Get image collection

In [4]:
# first get today's date
from datetime import date

today = date.today()
d1 = today.strftime("%Y-%m-%d")

collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
  .filterDate('2021-12-30', '2022-01-30') \
  .filterBounds(region) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 70)) \
  .median()
  #.map(maskS2clouds) \
  
# median = collection.median()

# Set viz parameters for three different layers

In [5]:
# true color
vizTC = {
    'bands' : ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 6000
}

vizSnow = {
    'bands': ['MSK_SNWPRB'],
    'min': 0,
    'max': 1,
    'palette': ['FFFFFF', 'FF0000']
}

vizFire = {
    'bands': ['B12', 'B8', 'B4'],
    'min': 600,
    'max': 2300
}

# Create an interactive map


In [6]:
Map = geemap.Map(add_google_map = False, layer_ctrl = True)
#center around region of interest
Map.centerObject(region, zoom = 9)

Map.addLayer(collection, vizTC, 'True Color')
Map.addLayer(collection, vizSnow, "Snow Probability")
Map.addLayer(collection, vizFire, "Wildfire")
Map



Map(center=[40.501466743410354, -105.53251000000024], controls=(WidgetControl(options=['position', 'transparen…

# Test out Landsat 8/9

In [93]:
land9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
 .filterDate('2020-01-01', d1) \
  .filterBounds(region)


In [74]:
geemap.image_props(land9.first()).getInfo()

{'ALGORITHM_SOURCE_SURFACE_REFLECTANCE': 'LaSRC_1.5.0',
 'ALGORITHM_SOURCE_SURFACE_TEMPERATURE': 'st_1.3.0',
 'CLOUD_COVER': 15.63,
 'CLOUD_COVER_LAND': 15.63,
 'COLLECTION_CATEGORY': 'T1',
 'COLLECTION_NUMBER': 2,
 'DATA_SOURCE_AIR_TEMPERATURE': 'MODIS',
 'DATA_SOURCE_ELEVATION': 'GLS2000',
 'DATA_SOURCE_OZONE': 'MODIS',
 'DATA_SOURCE_PRESSURE': 'Calculated',
 'DATA_SOURCE_REANALYSIS': 'GEOS-5 FP-IT',
 'DATA_SOURCE_WATER_VAPOR': 'MODIS',
 'DATE_ACQUIRED': '2021-11-12',
 'DATE_PRODUCT_GENERATED': 1642580930000,
 'DATUM': 'WGS84',
 'EARTH_SUN_DISTANCE': 0.9897287,
 'ELLIPSOID': 'WGS84',
 'GEOMETRIC_RMSE_MODEL': 6.349,
 'GEOMETRIC_RMSE_MODEL_X': 4.044,
 'GEOMETRIC_RMSE_MODEL_Y': 4.895,
 'GEOMETRIC_RMSE_VERIFY': 3.364,
 'GRID_CELL_SIZE_REFLECTIVE': 30,
 'GRID_CELL_SIZE_THERMAL': 30,
 'GROUND_CONTROL_POINTS_MODEL': 727,
 'GROUND_CONTROL_POINTS_VERIFY': 200,
 'GROUND_CONTROL_POINTS_VERSION': 5,
 'IMAGE_DATE': '2021-11-12',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'L1_DATE_PRODUCT

In [75]:
land9.size().getInfo()

20

In [129]:
dateOfInterest = datetime.strptime("2022-04-28", '%Y-%m-%d').timestamp()*1000
print(dateOfInterest)

1651125600000.0


## Write function to sort collection based on time (miliseconds) each image is from user specified date


In [130]:
def date_fun(image):
    return image.set(
    'dateDist',
    ee.Number(image.get('system:time_start')).subtract(dateOfInterest).abs()
    )
    

## Test out function

In [120]:
ee.Number(land9.first().get('system:time_start')).subtract(dateOfInterest).abs().getInfo()


1635087564035

In [135]:
land9 = land9.map(date_fun).sort('dateDist')

Now check, based on date of interest the first image in land9 should now be the oldest date

In [136]:
geemap.image_props(land9.first()).get('DATE_ACQUIRED').getInfo()


'2022-04-27'

In [106]:
# check different images to see if sorted correctly
land9List = land9.toList(land9.size())
ee.Image(land9List.get(1)).getInfo()

{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7701, 7891],
   'crs': 'EPSG:32613',
   'crs_transform': [30, 0, 328485, 0, -30, 4582815]},
  {'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7701, 7891],
   'crs': 'EPSG:32613',
   'crs_transform': [30, 0, 328485, 0, -30, 4582815]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7701, 7891],
   'crs': 'EPSG:32613',
   'crs_transform': [30, 0, 328485, 0, -30, 4582815]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7701, 7891],
   'crs': 'EPSG:32613',
   'crs_transform': [30, 0, 328485, 0, -30, 4582815]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
 

In [124]:
#print all values of dateDist

land9.aggregate_array('dateDist').getInfo()


[1635087564035,
 1635951859327,
 1636988707546,
 1637765930709,
 1638371100296,
 1639148329778,
 1639753499393,
 1640530726448,
 1641913128809,
 1642518298538,
 1643295523909,
 1643900690041,
 1644677913632,
 1645283085054,
 1646060312329,
 1646665479934,
 1647442708785,
 1648047876488,
 1648825098434,
 1649430268470]

In [134]:
land9.aggregate_array('DATE_ACQUIRED').getInfo()

['2022-04-27',
 '2022-04-20',
 '2022-04-11',
 '2022-04-04',
 '2022-03-26',
 '2022-03-19',
 '2022-03-10',
 '2022-03-03',
 '2022-02-22',
 '2022-02-15',
 '2022-02-06',
 '2022-01-30',
 '2022-01-14',
 '2022-01-05',
 '2021-12-29',
 '2021-12-20',
 '2021-12-13',
 '2021-12-04',
 '2021-11-22',
 '2021-11-12']

## Next Steps
<br>
    - Read in collection <br>
    - (maybe future step) Combine Landsat/Sentinel collections <br>
        OR filter each of them independently and choose one with smallest dateDist<br>
    - Apply date_fun to each collection and sort <br>
    - choose image with smallest dateDist and add that to map <br>
      